# Fitting of TCV q-Profiles

This notebook performs polynomial and piecewise linear fits of the safety factor (q) profiles for TCV plasmas, and compares the resulting fits for q-profile, magnetic shear, and shift. The workflow includes visualization and extraction of fit coefficients for further use.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.piecewise_qprofile_generator_inc import Context, run_qprofile_workflow, compare_PT_NT

## Resolution and num domain

In [ ]:
Nx = 48
Npieces = 8
x_in = 0.04
x_out = 0.08

## Positive triangularity configuration

In [ ]:
def qprofile_TCV_PT(r, ctx):
    y = r + ctx.R_axis
    a, b, c, d = 497.3420166252413, -1408.736172826569, 1331.4134861681464, -419.00692601227627
    return (a*y**3 + b*y**2 + c*y + d)

ctx_PT = Context(
    a_shift=0.25,
    Z_axis=0.1414361745,
    R_axis=0.8727315068,
    B_axis=1.4,
    R_LCFSmid=1.0968432365089495,
    qfunc=qprofile_TCV_PT,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
)
PT_qprof_data = run_qprofile_workflow(ctx_PT, qprofile_TCV_PT, return_data=True)

### Profile analysis

In [ ]:
run_qprofile_workflow(ctx_PT, qprofile_TCV_PT, plot=True)

### C Code for piecewise linear representation

In [ ]:
run_qprofile_workflow(ctx_PT, qprofile_TCV_PT, print_code=True)

## Negative triangularity configuration

In [ ]:
def qprofile_TCV_NT(r, ctx):
    y = r + ctx.R_axis
    a, b, c, d = 484.0615913225881, -1378.25993228584, 1309.3099150729233, -414.13270311478726
    return (a*y**3 + b*y**2 + c*y + d)

ctx_NT = Context(
    a_shift=0.5,
    Z_axis=0.1414361745,
    R_axis=0.8867856264,
    B_axis=1.4,
    R_LCFSmid=1.0870056099999,
    qfunc=qprofile_TCV_NT,
    x_inner=x_in,
    x_outer=x_out,
    Nx=Nx,
    Npieces=Npieces,
)

NT_qprof_data = run_qprofile_workflow(ctx_NT, qprofile_TCV_NT, return_data=True)

In [ ]:
run_qprofile_workflow(ctx_NT, qprofile_TCV_NT, plot=True)

### C Code for piecewise linear representation

In [ ]:
run_qprofile_workflow(ctx_NT, qprofile_TCV_NT, print_code=True)

## Comparison

In [ ]:
compare_PT_NT(PT_qprof_data, NT_qprof_data)